# LSTM 

### Importing the module that scrapes the internet for the latest values count

In [8]:
import csv_updater

In [9]:
conda install keras

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed

Note: you may need to restart the kernel to use updated packages.



Examining python:  28%|##7       | 27/98 [00:00<00:00, 27533.72it/s]

Comparing specs that have this dependency:   0%|          | 0/2 [00:00<?, ?it/s]


Finding conflict paths:   0%|          | 0/2 [00:00<?, ?it/s]


Finding shortest conflict path for python=3.8:   0%|          | 0/2 [00:00<?, ?it/s]


Finding shortest conflict path for python[version='>=3.8,<3.9.0a0']:  50%|#####     | 1/2 [00:00<?, ?it/s]


                                                                                                          


Finding conflict paths:   0%|          | 0/11 [00:00<?, ?it/s]


Finding shortest conflict path for python[version='>=3.7,<3.8.0a0']:   0%|          | 0/11 [00:00<?, ?it/s]


Finding shortest conflict path for python[version='>=3.5']:   9%|9         | 1/11 [00:00<00:00, 45.58it/s] 


Finding shortest conflict path for msgpack-python:  18%|#8        | 2/11 [00:07<00:33,  3.75s/it]         


Finding shortest conflict path for msgpack-python:  27%|##7       | 3/11 [00:07<00:

python=3.8 -> pip -> setuptools -> wincertstore[version='>=0.2']
keras -> python[version='>=3.6,<3.7.0a0'] -> pip -> setuptools -> wincertstore[version='>=0.2']
Package setuptools conflicts for:
keras -> python[version='>=3.6,<3.7.0a0'] -> pip -> setuptools[version='>=41.0.0']
keras -> tensorflow[version='<2.0'] -> tensorboard[version='>=2.0.0'] -> google-auth-oauthlib[version='>=0.4.1,<0.5'] -> requests-oauthlib[version='>=0.7.0'] -> oauthlib[version='>=0.6.2'] -> pyjwt[version='>=1.0.0'] -> flake8 -> setuptools[version='>=30.0.0']
python=3.8 -> pip -> setuptools
keras -> tensorflow[version='<2.0'] -> tensorboard[version='>=2.0.0'] -> google-auth[version='>=1.6.3,<2'] -> setuptools[version='>=40.3.0']
Package wheel conflicts for:
python=3.8 -> pip -> wheel
keras -> python[version='>=3.6,<3.7.0a0'] -> pip -> wheel
Package msgpack-python conflicts for:
keras -> python[version='>=3.6,<3.7.0a0'] -> pip -> cachecontrol -> msgpack-python



In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import  keras
from keras.preprocessing.sequence import TimeseriesGenerator

ModuleNotFoundError: No module named 'keras'

#### Getting the starter data from John Hopkin's CSV File

In [ ]:
df = pd.read_csv("johnhopkins-3March.csv")

In [ ]:
df = df.T

In [ ]:
cols_drop = []
for i in range(114):
    if i != 0 and i != 5:
        cols_drop.append(i)
df.drop(df.columns[cols_drop], axis = 1, inplace = True) 

In [ ]:
df.drop(['Country/Region', 'WHO region', 'Province/States'], inplace = True)

#### Cleaning the Data

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.rename(columns = {0:'Infected', 5:'Deaths'}, inplace = True)

In [ ]:
df.head()

In [ ]:
df.shape

#### Using the dates as indices

In [ ]:
dates_unix = []
start = 1579478400
for i in range(47):
    start += 86400
    dates_unix.append(start)

print(dates_unix)

In [ ]:
df['Date'] = dates_unix

In [ ]:
df.head()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], unit='s')

In [ ]:
df.head()

In [ ]:
df.set_index = df['Date']

In [ ]:
df_dates = df['Date']

In [ ]:
start

In [ ]:
df.head()

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.set_index("Date", inplace = True) 

In [ ]:
df.head()

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
df.head()

### Plotting the data

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(df.Infected, 'b--')
plt.plot(df.Deaths, 'r--')
plt.ylabel("Count")
plt.xlabel("Date")
plt.show()

In [ ]:
infected, deaths = csv_updater.get_nums()

In [ ]:
import datetime
now = datetime.datetime.now()

In [ ]:
currdate = now.strftime("%Y-%m-%d")

In [ ]:
infected, deaths

In [ ]:
# currdate = '2020-03-06'
currdate = pd.to_datetime(currdate)

In [ ]:
df.loc[currdate]= [infected, deaths]

In [ ]:
df.tail(1)

In [ ]:
if df.loc[currdate].any():
    print("Data already exists")    
else:
    df.loc[currdate] = [infected, deaths]

In [ ]:
df.tail()

In [ ]:
df.loc['2020-03-04'] = [95310,3285]

In [ ]:
df.loc['2020-03-05'] = [98414,3387]

In [ ]:
# df.drop(['2020-03-04','2020-03-05'], inplace=True)

### Exporting the well structured data

In [ ]:
df.to_csv("StructuredDataset.csv")

In [ ]:
df = pd.read_csv("StructuredDataset.csv")

In [ ]:
df.set_index('Date', inplace = True)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(df.Infected, 'b--')
plt.plot(df.Deaths, 'r--')
plt.xlabel('Dates')
plt.show()

## LSTM

In [ ]:
df

In [ ]:
df.drop("2020-03-08", inplace=True)

In [ ]:
df

In [ ]:
df_inf = df.iloc[:,0]

In [ ]:
df_inf['2020-01-21']

In [ ]:
df_dea = df.iloc[:,1]

### Infected Predictions

In [ ]:
df_inf = df_inf.values
df_inf = df_inf.reshape((-1,1))

In [ ]:
split_percent = 0.80
split = int(split_percent*len(df_inf))

inf_train = df_inf[:split]
inf_test = df_inf[split:]

In [ ]:
date_train = df[:split]
date_test = df['Date'][split:]

In [ ]:
look_back = 9

train_generator = TimeseriesGenerator(inf_train, inf_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(inf_test, inf_test, length=look_back, batch_size=1)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

In [ ]:
prediction = model.predict_generator(test_generator)

inf_train = inf_train.reshape((-1))
inf_test = inf_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()